In [38]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import train_test_split
import emoji

In [44]:
def load_data(file_path):
    """Load the Twitter data from CSV"""
    df = pd.read_csv('data/data.csv', encoding='utf-8')
    # Keep only English tweets
    df = df[df['Language'] == 'en']
    # Reset index after filtering
    df.reset_index(drop=True, inplace=True)
    return df


In [43]:
def clean_text(text):
    """Clean and preprocess tweet text"""
    if isinstance(text, str):
        # Convert to lowercase
        text = text.lower()
        
        # Remove URLs
        text = re.sub(r'https?://\S+|www\.\S+', '', text)
        
        # Replace emojis with their descriptions
        text = emoji.demojize(text)
        
        # Remove mentions (@username)
        text = re.sub(r'@\w+', '', text)
        
        # Extract hashtags but remove the # symbol
        hashtags = re.findall(r'#(\w+)', text)
        # Remove the hashtags from the text
        text = re.sub(r'#\w+', '', text)
        
        # Remove special characters and extra whitespace
        text = re.sub(r'[^\w\s]', ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        
        return text
    return ""



In [45]:
def extract_features(df):
    """Extract additional features from the data"""
    # Text length
    df['text_length'] = df['Tweet_Content'].apply(lambda x: len(str(x)) if isinstance(x, str) else 0)
    
    # Count of hashtags
    df['hashtag_count'] = df['Tweet_Content'].apply(
        lambda x: len(re.findall(r'#\w+', str(x))) if isinstance(x, str) else 0
    )
    
    # Count of mentions
    df['mention_count'] = df['Tweet_Content'].apply(
        lambda x: len(re.findall(r'@\w+', str(x))) if isinstance(x, str) else 0
    )
    
    # Uppercase ratio (possible indicator of shouting/aggression)
    df['uppercase_ratio'] = df['Tweet_Content'].apply(
        lambda x: sum(1 for c in str(x) if c.isupper())/len(str(x)) if isinstance(x, str) and len(str(x)) > 0 else 0
    )
    
    # Check for controversial keywords (customize this list based on your domain)
    troll_keywords = ['terrorist', 'nazi', 'evil', 'kill', 'murder', 'destroy', 'genocide', 'apartheid']
    df['controversial_keyword_count'] = df['Tweet_Content'].apply(
        lambda x: sum(str(x).lower().count(word) for word in troll_keywords) if isinstance(x, str) else 0
    )
    
    # Engagement features that may indicate viral/controversial content
    df['engagement_score'] = df['Reply_Count'] + df['Repost_Count'] + df['Like_Count']
    
    # Verified status as a numeric feature
    df['is_verified'] = df['Verified_Status'].apply(lambda x: 1 if x == 'True' else 0)
    
    return df

In [46]:
def create_troll_labels(df):
    """
    Create labels for trolling detection (this is a simplified heuristic approach)
    You would ideally have human-labeled data or a more sophisticated approach
    """
    # This is a simplified labeling approach - in reality you'd want human labeling or better heuristics
    # Criteria: high controversial keywords, high uppercase ratio, and relatively high engagement
    
    # Normalize the features for scoring
    df['norm_controversial'] = df['controversial_keyword_count'] / df['controversial_keyword_count'].max()
    df['norm_uppercase'] = df['uppercase_ratio'] / df['uppercase_ratio'].max() 
    df['norm_engagement'] = np.log1p(df['engagement_score']) / np.log1p(df['engagement_score']).max()
    
    # Create a simple troll score
    df['troll_score'] = (df['norm_controversial'] * 0.5 + 
                         df['norm_uppercase'] * 0.3 + 
                         df['norm_engagement'] * 0.2)
    
    # Label as potential troll if score is above threshold
    # This threshold would need tuning
    threshold = df['troll_score'].quantile(0.75)  # Top 25% as trolls for balanced dataset
    df['troll_label'] = (df['troll_score'] > threshold).astype(int)
    
    return df

In [47]:
def prepare_dataset(df):
    """Prepare the final dataset for the model"""
    # Clean the tweet content
    df['cleaned_text'] = df['Tweet_Content'].apply(clean_text)
    
    # Extract features
    df = extract_features(df)
    
    # Create labels
    df = create_troll_labels(df)
    
    # Select relevant columns for modeling
    features = [
        'cleaned_text', 'text_length', 'hashtag_count', 'mention_count',
        'uppercase_ratio', 'controversial_keyword_count', 'engagement_score',
        'is_verified', 'troll_label'
    ]
    
    return df[features]


In [49]:
def main():
    # File path to your data
    file_path = 'data/data.csv'
    
    # Load and process data
    df = load_data(file_path)
    processed_df = prepare_dataset(df)
    
    # Save the processed data
    processed_df.to_csv('data/processed_twitter_data.csv', index=False)
    
    # Split into train and test sets
    X = processed_df.drop('troll_label', axis=1)
    y = processed_df['troll_label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Save train and test sets
    pd.concat([X_train, y_train], axis=1).to_csv('data/train_data.csv', index=False)
    pd.concat([X_test, y_test], axis=1).to_csv('data/test_data.csv', index=False)
    
    print(f"Processing complete. Generated {sum(y)} potential troll instances out of {len(y)} total records.")
    print(f"Processed data saved to '../data/processed_twitter_data.csv'")
    print(f"Train/test splits saved to '../data/train_data.csv' and '../data/test_data.csv'")

if __name__ == "__main__":
    main()

Processing complete. Generated 134 potential troll instances out of 537 total records.
Processed data saved to '../data/processed_twitter_data.csv'
Train/test splits saved to '../data/train_data.csv' and '../data/test_data.csv'


In [50]:
import pandas as pd
import random

def create_labeling_sample(df, sample_size=200):
    """Create a sample for manual labeling"""
    # Sample tweets for manual labeling
    sampled_df = df.sample(sample_size, random_state=42)
    
    # Add columns for manual labeling
    sampled_df['is_troll'] = None
    sampled_df['troll_category'] = None
    sampled_df['notes'] = None
    
    # Save the sample for manual labeling
    sampled_df.to_csv('../data/manual_labeling_sample.csv', index=False)
    
    return sampled_df



In [51]:
import pandas as pd
import random

def create_labeling_sample(df, sample_size=200):
    """Create a sample for manual labeling"""
    # Sample tweets for manual labeling
    sampled_df = df.sample(sample_size, random_state=42)
    
    # Add columns for manual labeling
    sampled_df['is_troll'] = None
    sampled_df['troll_category'] = None
    sampled_df['notes'] = None
    
    # Save the sample for manual labeling
    sampled_df.to_csv('../data/manual_labeling_sample.csv', index=False)
    
    return sampled_df

In [52]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np

In [53]:
def build_text_sequence_model(max_words=10000, max_sequence_length=100):
    """Build a sequential model for text classification"""
    model = Sequential([
        # Embedding layer
        Embedding(input_dim=max_words, output_dim=128, input_length=max_sequence_length),
        
        # Bidirectional LSTM layers
        Bidirectional(LSTM(64, return_sequences=True)),
        Dropout(0.3),
        Bidirectional(LSTM(32)),
        Dropout(0.3),
        
        # Dense layers
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')  # Binary classification
    ])

In [55]:
# Build the model
model = build_text_sequence_model()

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

# Return the model (if needed)
model


C:\Users\mahdi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


AttributeError: 'NoneType' object has no attribute 'compile'